In [1]:
# import sys
# sys.path.insert(0, '..')
from semantic_mpc_interface import (
    LoadModel,
    get_thermostat_data,
    HPFlexSurvey,
    convert_units,
    SHACLHandler,
    inline_shapes
    # add_connection
)
from buildingmotif.namespaces import BRICK, RDF, S223
from rdflib import URIRef
from buildingmotif import BuildingMOTIF
from buildingmotif.dataclasses import Library
import csv
from pyshacl.rdfutil import clone

# SELECT ONTOLOGY 
ontology = 's223'

# base path
base_path = f'{ontology}-test_site/test_build'

CRITICAL:root:Install the 'bacnet-ingress' module, e.g. 'pip install buildingmotif[bacnet-ingress]'


# Testing the Model Builder

ModelBuilder has been deprecated in favor of just survey based workflow. May be readded in the future

# Testing the Survey Workflow

In [2]:
# Please disregard excessive outputs (logging and warnings)
# TODO: Figure out how to configure these in building motif 
import logging
logging.disable(logging.CRITICAL)
import warnings
warnings.filterwarnings("ignore")

In [3]:
# Creating survey, allow overwrite if there is something already there
s = HPFlexSurvey(base_path.split('/')[0],base_path.split('/')[1],'.', overwrite=True, ontology=ontology,
                 template_dict = {'zone': 'hvac-zone',
                                  "space": "space",
                                  "hvac": "hp-rtu",
                                  "tstat": "tstat",
                                  "window": "window",
                                  "site": "site" })

# Generating a simple building structure that prefills csv files. 
s.easy_config(zone_space_window_list=[(2,2),(1,2),(1,3)])
# Now check test_site/test_build to look at survey files


removing dependency:  space
removing dependency:  window
removing dependency:  hp-rtu
removing dependency:  tstat
values: dict_values([])
values: dict_values(['area-value'])
values: dict_values(['heating_COP-value', 'cooling_capacity-value', 'cooling_COP-value', 'heating_capacity-value'])
values: dict_values(['setpoint_deadband-value', 'tolerance-value', 'stage_count-value', 'resolution-value', 'active-value'])
values: dict_values(['azimuth-value', 'tilt-value', 'area-value'])
values: dict_values(['noaastation-value', 'timezone-value', 'longitude-value', 'latitude-value'])
{'site_id': 's223-test_site', 'hvac_type': 'hp-rtu', 'variable_params': {'hvac-zone': {}, 'space': {'area': '<name>-area'}, 'hp-rtu': {'cooling_COP': '<name>-cooling_COP', 'heating_capacity': '<name>-heating_capacity', 'cooling_capacity': '<name>-cooling_capacity', 'heating_COP': '<name>-heating_COP'}, 'tstat': {'resolution': '<name>-resolution', 'setpoint_deadband': '<name>-setpoint_deadband', 'tolerance': '<name>-t

In [4]:
# Will just fill the columns programmatically for testing, csv should be filled out otherwise
import sys
sys.path.insert(0,'../examples')
from example_prefill_usage import prefill_csv_survey

In [5]:
prefill_csv_survey(base_path)

Loading config from: s223-test_site/test_build/config.json
Found CSV files: ['zone', 'site', 'window', 'hvac', 'point_list', 'tstat', 'space']
Filled 10 empty cells in zone.csv
Filled 4 empty cells across 4 columns in site.csv
Filled 21 empty cells across 3 columns in window.csv
Filled 12 empty cells across 4 columns in hvac.csv
No empty cells found in point_list.csv
Filled 15 empty cells across 5 columns in tstat.csv
Filled 4 empty cells across 1 columns in space.csv


In [6]:
# Reading csv
s.read_csv()

Expanded CSV shape: (6, 5)
New columns added: []
Expanded CSV shape: (4, 4)
New columns added: ['area']
Expanded CSV shape: (3, 13)
New columns added: ['cooling_COP', 'heating_capacity', 'cooling_capacity', 'heating_COP']
Expanded CSV shape: (3, 14)
New columns added: ['resolution', 'setpoint_deadband', 'tolerance', 'stage_count', 'active']
Expanded CSV shape: (7, 10)
New columns added: ['area', 'azimuth', 'tilt']
Expanded CSV shape: (1, 9)
New columns added: ['longitude', 'noaastation', 'timezone', 'latitude']


# Testing SHACL Generation and Validation

In [7]:
og = clone.clone_graph(s.graph)
# Create handler
handler = SHACLHandler(ontology=ontology)

# Generate shapes
handler.generate_shapes()

# Save shapes
handler.save_shapes(f'{base_path}/shapes.ttl')
# also save more human readable shapes
inline_shapes(handler.shapes_graph).serialize(f"{base_path}/inlined_shapes.ttl")

# Run inference on model
inferred_graph = handler.infer(s.graph)

inferred_graph.serialize(f'{base_path}/reasoned.ttl', format = 'ttl')
# if not validation_result.valid:
#     print("Validation failed:")
#     print(validation_result.report_string)

<Graph identifier=a5cb9957-be13-46fb-9b81-614afaa440cc (<class 'rdflib.graph.Graph'>)>

In [8]:
# lots of new inferred information
(inferred_graph-og).print()

@prefix ns1: <urn:hpflex/shapes#> .

<urn:hpflex/s223-test_site#hvac_1> a ns1:hp-rtu ;
    ns1:air-connected-to <urn:hpflex/s223-test_site#zone_1> ;
    ns1:has-point <urn:hpflex/s223-test_site#hvac_1-cooling_COP>,
        <urn:hpflex/s223-test_site#hvac_1-cooling_capacity>,
        <urn:hpflex/s223-test_site#hvac_1-heating_COP>,
        <urn:hpflex/s223-test_site#hvac_1-heating_capacity> .

<urn:hpflex/s223-test_site#hvac_2> a ns1:hp-rtu ;
    ns1:air-connected-to <urn:hpflex/s223-test_site#zone_2> ;
    ns1:has-point <urn:hpflex/s223-test_site#hvac_2-cooling_COP>,
        <urn:hpflex/s223-test_site#hvac_2-cooling_capacity>,
        <urn:hpflex/s223-test_site#hvac_2-heating_COP>,
        <urn:hpflex/s223-test_site#hvac_2-heating_capacity> .

<urn:hpflex/s223-test_site#hvac_3> a ns1:hp-rtu ;
    ns1:air-connected-to <urn:hpflex/s223-test_site#zone_3> ;
    ns1:has-point <urn:hpflex/s223-test_site#hvac_3-cooling_COP>,
        <urn:hpflex/s223-test_site#hvac_3-cooling_capacity>,
        

# Testing get Metadata

In [9]:
# still working on loader, will clean up class, but functionality about right
loader = LoadModel(f"{base_path}/reasoned.ttl", 
                   template_dict={'sites':'site', 'zones': 'hvac-zone'}, 
                   ontology = ontology)
site_info = loader.get_all_building_objects()

In [10]:
print(site_info)

{'sites': [site(name='urn:hpflex/s223-test_site#s223-test_site', timezone=Value(value=1.0, unit='None'), noaastation=Value(value=1.0, unit='None'), longitude=Value(value=1.0, unit='http://qudt.org/vocab/unit/Degree'), latitude=Value(value=1.0, unit='http://qudt.org/vocab/unit/Degree'), )], 'zones': [hvac_zone(name='urn:hpflex/s223-test_site#zone_3', , windows=1, spaces=1, hp_rtus=1, tstats=1), hvac_zone(name='urn:hpflex/s223-test_site#zone_2', , windows=1, spaces=1, hp_rtus=1, tstats=1), hvac_zone(name='urn:hpflex/s223-test_site#zone_1', , windows=2, spaces=2, hp_rtus=1, tstats=1)]}


In [11]:
zone = site_info['zones'][0]
print(zone)

hvac_zone(name='urn:hpflex/s223-test_site#zone_3', , windows=1, spaces=1, hp_rtus=1, tstats=1)


In [12]:
site_info['zones'][-1].spaces

[space(name='urn:hpflex/s223-test_site#space_1_2', area=Value(value=2.0, unit='http://qudt.org/vocab/unit/FT2'), ),
 space(name='urn:hpflex/s223-test_site#space_1_1', area=Value(value=1.0, unit='http://qudt.org/vocab/unit/FT2'), )]

In [13]:
zone.windows

[window(name='urn:hpflex/s223-test_site#window_3_3', tilt=Value(value=7.0, unit='http://qudt.org/vocab/unit/Degree'), azimuth=Value(value=7.0, unit='http://qudt.org/vocab/unit/Degree'), area=None, )]

In [14]:
zone.tstats[0]

tstat(name='urn:hpflex/s223-test_site#tstat_zone_3', tstat_tolerance=Value(value=3.0, unit='http://qudt.org/vocab/unit/DEG_F'), tstat_stage_count=Value(value=3.0, unit='http://qudt.org/vocab/unit/NUM'), tstat_deadband=Value(value=3.0, unit='http://qudt.org/vocab/unit/DEG_F'), tstat_resolution=None, tstat_active=Value(value=3.0, unit='http://qudt.org/vocab/unit/NUM'), )

In [15]:
# optionally just load everything as si 
si_loader = LoadModel(f"{base_path}/reasoned.ttl", 
                   template_dict={'sites':'site', 'zones': 'hvac-zone'}, 
                   ontology = ontology,
                   as_si_units=True)
site_info = si_loader.get_all_building_objects()
print(zone.tstats[0].tstat_resolution)

None


In [16]:
# Getting the thermostat metadata
get_thermostat_data(si_loader)

{'heat_availability': [True, True, True],
 'cool_availability': [True, True, True],
 'heat_tolerance': [],
 'cool_tolerance': [],
 'setpoint_deadband': [],
 'active': [],
 'control_group': ['DEPRECATED', 'DEPRECATED', 'DEPRECATED'],
 'control_type_list': [],
 'floor_area_list': [0.65032128, 0.37161216, 0.27870912000000003],
 'floor_area_unit': ['http://qudt.org/vocab/unit/M2',
  'http://qudt.org/vocab/unit/M2',
  'http://qudt.org/vocab/unit/M2'],
 'window_area_list': [0.0, 0.0, 0.0],
 'window_area_unit': ['Unknown', 'Unknown', 'Unknown'],
 'azimuth_list': [None, None, None],
 'azimuth_unit': ['Unknown', 'Unknown', 'Unknown'],
 'tilt_list': [None, None, None],
 'tilt_unit': ['Unknown', 'Unknown', 'Unknown'],
 'zone_ids': ['zone_3', 'zone_2', 'zone_1'],
 'hvacs': ['unknown', 'unknown', 'unknown'],
 'setpoint_type': ['double', 'double', 'double'],
 'fuel_heat_list': ['electricity', 'electricity', 'electricity'],
 'fuel_cool_list': ['electricity', 'electricity', 'electricity'],
 'cooling_c

In [17]:
# Getting the thermostat metadata for 1 zone 
get_thermostat_data(si_loader, ['zone_1','zone_2'])

{'heat_availability': [True, True],
 'cool_availability': [True, True],
 'heat_tolerance': [],
 'cool_tolerance': [],
 'setpoint_deadband': [],
 'active': [],
 'control_group': ['DEPRECATED', 'DEPRECATED'],
 'control_type_list': [],
 'floor_area_list': [0.37161216, 0.27870912000000003],
 'floor_area_unit': ['http://qudt.org/vocab/unit/M2',
  'http://qudt.org/vocab/unit/M2'],
 'window_area_list': [0.0, 0.0],
 'window_area_unit': ['Unknown', 'Unknown'],
 'azimuth_list': [None, None],
 'azimuth_unit': ['Unknown', 'Unknown'],
 'tilt_list': [None, None],
 'tilt_unit': ['Unknown', 'Unknown'],
 'zone_ids': ['zone_2', 'zone_1'],
 'hvacs': ['unknown', 'unknown'],
 'setpoint_type': ['double', 'double'],
 'fuel_heat_list': ['electricity', 'electricity'],
 'fuel_cool_list': ['electricity', 'electricity'],
 'cooling_capacity': [],
 'cooling_capacity_unit': [],
 'heating_capacity': [],
 'heating_capacity_unit': [],
 'cooling_cop': [],
 'heating_cop': [],
 'cooling_electricity': [True, True],
 'heati

# Testing Unit Conversion

In [18]:
convert_units(10, 'FT', 'M')

3.048

In [19]:
convert_units(0, 'DEG_C', 'DEG_F')

31.999999999999886

In [20]:
convert_units(0, 'DEG_C', 'K')

273.15

In [21]:
# Planning to implement converting everything to si in loader
# loader.get_all_building_objects

# Testing Grafana Dashboarding 
Needs updates

In [22]:
# import yaml 
# with open('../development_files/creds.yml') as f:
#     config = yaml.safe_load(f)

# bg = BrickToGrafana(grafana_server=config['grafana_server'], grafana_api_key = config['grafana_api_key'], datasource=config['datasource'], ttl_path = 'test-brick-model.ttl')

In [23]:
# bg.create_dashboard('AQL from brick')

In [24]:
# Grafana not set up, upload won't work
# bg.upload_dashboard(message = 'testing upload')